In [5]:
import numpy as np
import pandas as pd
import os
import sys
import torch
import csv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import optuna
import random
import joblib
from config import RESULTS_DIR, IMAGES_DIR, DATA_DIR, DATASET_PATH, MOTIF_INDEXES_PATH, K, N, P, NORMALIZE_FLAGS

print(f"Results will be saved in: {RESULTS_DIR}")
print(f"Images will be saved in: {IMAGES_DIR}")
print(f"Data will be accessed from: {DATA_DIR}")

os.makedirs(RESULTS_DIR, exist_ok=True)

if '__file__' in globals():
    # For standalone scripts
    base_dir = os.path.dirname(__file__)
else:
    # For Jupyter or interactive environments
    base_dir = os.getcwd()

# Add the parent directory of `utils` to the Python path
sys.path.append(os.path.abspath(os.path.join(base_dir, "../")))

Results will be saved in: /home/mgsilva/motifpred/results/syntheticdata1/variables=[0,2]
Images will be saved in: /home/mgsilva/motifpred/images/syntheticdata1/variables=[0,2]
Data will be accessed from: /home/mgsilva/motifpred/data/syntheticdata1/variables=[0,2]


In [6]:
data = np.genfromtxt(DATASET_PATH, delimiter=",").astype(int).reshape((K, N))
motif_indexes = np.genfromtxt(MOTIF_INDEXES_PATH, delimiter=",").astype(int)

print(motif_indexes)

[    0    10    22 ... 99922 99956 99992]


In [7]:
# Import shared setup
from utils.setup import seed, device, early_stopper, pipeline, test_tensor

# Example usage
print(f"Device: {device}")
test_tensor()

Device: cuda
tensor([[0.5035, 0.9978, 0.3884],
        [0.6929, 0.1703, 0.1384],
        [0.4759, 0.7481, 0.0361],
        [0.5062, 0.8469, 0.2588],
        [0.2707, 0.4115, 0.6839]])


In [8]:
from utils.utils import create_dataset

lookback_period = 100 #window size
step = 5 #step size for the sliding window
forecast_period = 50 #forward window size

#X_series: past window, X_indices: indexes of the motif in the window,  y: next relative index of the motif
X_series, X_indices, X_mask, y = create_dataset(data, lookback_period, step, forecast_period, motif_indexes, P)

# X_series, X2, and y are now PyTorch tensors
print("X_series shape:", X_series.shape)  # Expected shape: (num_samples, lookback_period, num_features)
print("X_mask shape:", X_mask.shape)  # Expected shape: (num_samples, lookback_period)
print("X_indices shape:", X_indices.shape)  # Expected shape: (num_samples, max_motif_length_in_window)
print("y shape:", y.shape)    # Expected shape: (num_samples, 1)


X_series shape: torch.Size([19979, 100, 3])
X_mask shape: torch.Size([19979, 100])
X_indices shape: torch.Size([19979, 6, 1])
y shape: torch.Size([19979, 1])


In [9]:
from models.ffnn_pytorch import FFNN
from models.lstm_pytorch import LSTM
from models.cnn_pytorch import CNN
from models.tcn_pytorch import TemporalConvNet
from models.transformer_pytorch import TimeSeriesTransformer
from models.baseline_pytorch import BaselineAverage, BaselineLastDifference
from utils.utils import print_study_results, get_best_model_results, plot_best_model_results, plot_preds_vs_truevalues
from utils.train_pipeline import get_preds_best_config

models = ["FFNN", "LSTM", "CNN", "TCN", "Transformer", "Baseline"]
inputs = ["Series", "Series_Masking", "Indexes"]

n_trials = 100
num_epochs = 500


def process_baseline_model(model_class, input_name, X, normalize_flags, n_trials, num_epochs, pipeline, seed, y):
    """Process baseline models."""
    model_name = f"{model_class.__name__}{input_name}"
    dir = os.path.join(RESULTS_DIR, f"{model_name}_{n_trials}_trials_{num_epochs}_epochs")

    if os.path.exists(os.path.join(dir, "best_model_results.csv")):
        print(f"Model {model_name} already exists")
        return

    study = joblib.load(os.path.join(dir, "study.pkl"))
    fold_val_losses, fold_test_losses = get_best_model_results(study)

    (epochs_train_losses, epochs_val_losses, val_losses, test_losses,
     test_mae_per_fold, test_rmse_per_fold, all_predictions, all_true_values) = get_preds_best_config(
        study, pipeline, model_class, "Baseline", [], num_epochs=num_epochs, seed=seed, X=X, y=y, normalize_flags=normalize_flags
    )

    if not np.allclose(fold_val_losses, val_losses):
        raise Exception("Best model val losses are not close to val losses")

    # Save results to CSV
    results = pd.DataFrame({
        "fold": np.arange(1, 6),
        "val_loss": fold_val_losses,
        "test_loss": fold_test_losses,
        "test_mae": test_mae_per_fold,
        "test_rmse": test_rmse_per_fold
    })
    results.to_csv(os.path.join(dir, "best_model_results.csv"), index=False, mode='w')


def process_non_baseline_model(model_type, model_params_keys, input_name, X, normalize_flags, num_epochs, seed, pipeline, y):
    """Process non-baseline models."""
    model_name = f"{model_type}{input_name}"
    model_results_dir = os.path.join(RESULTS_DIR, f"{model_name}_{n_trials}_trials_{num_epochs}_epochs")

    if os.path.exists(os.path.join(model_results_dir, "best_model_results.csv")):
        print(f"Model {model_name} already exists")
        return

    study = joblib.load(os.path.join(model_results_dir, "study.pkl"))
    fold_val_losses, fold_test_losses = get_best_model_results(study)

    if model_type == "Transformer":
        model = eval("TimeSeriesTransformer")
    elif model_type == "TCN":
        model = eval("TemporalConvNet")
    else:
        model = eval(model_type)

    (epochs_train_losses, epochs_val_losses, val_losses, test_losses,
     test_mae_per_fold, test_rmse_per_fold, all_predictions, all_true_values) = get_preds_best_config(
        study, pipeline, model, model_type, model_params_keys, num_epochs=num_epochs, seed=seed, X=X, y=y, normalize_flags=normalize_flags
    )

    #if not np.allclose(fold_val_losses, val_losses, atol=1):
    #    raise Exception("Best model val losses are not close to val losses")

    # Save results to CSV
    results = pd.DataFrame({
        "fold": np.arange(1, 6),
        "val_loss": fold_val_losses,
        "test_loss": fold_test_losses,
        "test_mae": test_mae_per_fold,
        "test_rmse": test_rmse_per_fold
    })
    results.to_csv(os.path.join(model_results_dir, "best_model_results.csv"), index=False, mode='w')


for model_type in models:
    for input_name in inputs:
        normalize_flags = NORMALIZE_FLAGS
        n_trials, num_epochs = (1, 1) if model_type == "Baseline" else (n_trials, num_epochs)

        if model_type == "Baseline":
            if input_name != "Indexes":
                continue

            X = {"X_series": X_series, "X_mask": X_mask, "X_indices": X_indices}
            normalize_flags = {"X_series": True, "X_mask": False, "X_indices": False}

            for model_class in [BaselineAverage, BaselineLastDifference]:
                process_baseline_model(
                    model_class, input_name, X, normalize_flags, n_trials, num_epochs, pipeline, seed, y
                )

        else:
            if input_name == "Series":
                X = {"X_series": X_series}
            elif input_name == "Series_Masking":
                X = {"X_series": X_series, "X_mask": X_mask}
            else:
                X = {"X_indices": X_indices}


            model_params_map = {
                "FFNN": ["hidden_sizes_list"],
                "LSTM": ["hidden_sizes_list"],
                "CNN": ["kernel_size", "num_filters_list", "pool_size"],
                "TCN": ["kernel_size", "num_channels_list", "dropout"],
                "Transformer": ["d_model", "n_heads", "e_layers", "dim_feedforward", "dropout"]
            }

            process_non_baseline_model(
                model_type, model_params_map[model_type], input_name, X, normalize_flags, num_epochs, seed, pipeline, y
            )


Model FFNNSeries already exists
Model FFNNSeries_Masking already exists
Model FFNNIndexes already exists
Model LSTMSeries already exists
Model LSTMSeries_Masking already exists
Model LSTMIndexes already exists
Model CNNSeries already exists
Model CNNSeries_Masking already exists
Model CNNIndexes already exists
Model TCNSeries already exists
Best hyperparameters: {'learning_rate': 0.0006119580363897532, 'kernel_size': 7, 'receptive_field': 50, 'dropout': 7.786373649315559e-05, 'batch_size': 32, 'block_channels_0': 32, 'block_channels_1': 32, 'block_channels_2': 32, 'block_channels_3': 16}
Stopping early: Maximum training time exceeded.
Early stopping at epoch 108, with best epoch being 97
Stopping early: Maximum training time exceeded.
Early stopping at epoch 108, with best epoch being 98
Stopping early: Maximum training time exceeded.
Early stopping at epoch 102, with best epoch being 84
Early stopping at epoch 111, with best epoch being 100
Stopping early: Maximum training time exceed

FileNotFoundError: [Errno 2] No such file or directory: '/home/mgsilva/motifpred/results/syntheticdata1/variables=[0,2]/TCNIndexes_100_trials_500_epochs/study.pkl'

In [10]:
import numpy as np
from scipy import stats
rng = np.random.default_rng()
from scipy.stats import ttest_rel

models = ["FFNN", "LSTM", "CNN","TCN", "Transformer", "Baseline"]
inputs = ["Series", "Series_Masking", "Indexes"]

results_df = pd.DataFrame(columns=["model", "input", "fold", "mae", "rmse"])

# Combine each model with each input
for model_type in models:
    for input_name in inputs:
        # Handle baseline-specific logic
        if model_type == "Baseline":
            n_trials, num_epochs = (1, 1)
            if input_name != "Indexes":
                continue
            
            # Process both BaselineAverage and BaselineLastDifference
            baseline_variants = ["BaselineAverage", "BaselineLastDifference"]
            for baseline_type in baseline_variants:
                model_name = f"{baseline_type}{input_name}"
                print(f"Processing Model: {model_name}")

                # Construct the results directory path
                model_results_dir = os.path.join(RESULTS_DIR, f"{model_name}_{n_trials}_trials_{num_epochs}_epochs")
                results_file = os.path.join(model_results_dir, "best_model_results.csv")

                # Skip if results file doesn't exist
                if not os.path.exists(results_file):
                    print(f"Results file for {model_name} not found. Skipping.")
                    continue

                # Load results from CSV
                results = pd.read_csv(results_file)
                maes = results["test_mae"].values
                rmses = results["test_rmse"].values

                # Add results to the dataframe
                for i in range(len(maes)):  # Assuming results have folds
                    results_df = pd.concat([
                        results_df,
                        pd.DataFrame([{
                            "model": baseline_type,
                            "input": input_name,
                            "fold": i + 1,
                            "mae": maes[i],
                            "rmse": rmses[i]
                        }])
                    ], ignore_index=True)

        else:
            n_trials = 100
            num_epochs = 500

            model_name = f"{model_type}{input_name}"
            print(f"Processing Model: {model_name}")

            # Construct the results directory path
            model_results_dir = os.path.join(RESULTS_DIR, f"{model_name}_{n_trials}_trials_{num_epochs}_epochs")
            results_file = os.path.join(model_results_dir, "best_model_results.csv")

            # Skip if results file doesn't exist
            if not os.path.exists(results_file):
                print(f"Results file for {model_name} not found. Skipping.")
                continue

            # Load results from CSV
            results = pd.read_csv(results_file)
            maes = results["test_mae"].values
            rmses = results["test_rmse"].values

            # Add results to the dataframe
            for i in range(len(maes)):  # Assuming results have folds
                results_df = pd.concat([
                    results_df,
                    pd.DataFrame([{
                        "model": model_type,
                        "input": input_name,
                        "fold": i + 1,
                        "mae": maes[i],
                        "rmse": rmses[i]
                    }])
                ], ignore_index=True)

# Display the results
print(results_df)

Processing Model: FFNNSeries
Processing Model: LSTMSeries
Processing Model: CNNSeries
Processing Model: TCNSeries
Results file for TCNSeries not found. Skipping.
Processing Model: TransformerSeries
          model   input fold       mae       rmse
0          FFNN  Series    1  9.389045  11.669245
1          FFNN  Series    2  8.793365  10.960507
2          FFNN  Series    3  9.180077  11.325398
3          FFNN  Series    4  8.737524  11.029864
4          FFNN  Series    5  8.585275  10.890684
5          LSTM  Series    1  4.258052   7.083573
6          LSTM  Series    2  4.590631   7.886975
7          LSTM  Series    3  3.035272   6.043316
8          LSTM  Series    4  3.031424   6.217641
9          LSTM  Series    5  7.226361   9.748582
10          CNN  Series    1  6.470577   8.659925
11          CNN  Series    2  6.460783   8.527258
12          CNN  Series    3  5.694703   7.675818
13          CNN  Series    4  6.876115   9.207098
14          CNN  Series    5  6.587561   8.773660
15

/tmp/ipykernel_61335/153205014.py:76: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([


In [8]:
#average fold results for each model and input
avg_results_df = results_df.groupby(["model", "input"]).mean().reset_index()
print(avg_results_df)

                     model           input fold       mae       rmse
0          BaselineAverage         Indexes  3.0  7.061335   9.466859
1   BaselineLastDifference         Indexes  3.0  4.363204   7.537955
2                      CNN         Indexes  3.0  0.345493   0.555321
3                      CNN          Series  3.0  6.417948   8.568752
4                      CNN  Series_Masking  3.0  0.701390   0.967899
5                     FFNN         Indexes  3.0  0.549793   1.042796
6                     FFNN          Series  3.0  8.937057  11.175140
7                     FFNN  Series_Masking  3.0  1.012417   1.686616
8                     LSTM         Indexes  3.0  0.148808   0.186667
9                     LSTM          Series  3.0  4.428348   7.396018
10                    LSTM  Series_Masking  3.0  1.099881   2.904646
11             Transformer         Indexes  3.0  0.403696   0.533269
12             Transformer          Series  3.0  1.750024   3.790114
13             Transformer  Series

In [ ]:
for input_type in inputs:
    for metric in ["mae", "rmse"]:
        # Filter data for the selected input type and always include baseline models
        df_filtered = results_df[(results_df['input'] == input_type) | (results_df['model'].str.contains("Baseline"))]

        # Get the unique models
        models = df_filtered['model'].unique()

        # Initialize an empty matrix for p-values
        pval_matrix = pd.DataFrame(np.nan, index=models, columns=models)

        # Fill the matrix with p-values
        for i, model1 in enumerate(models):
            for j, model2 in enumerate(models):
                if i < j:  # Avoid redundant comparisons (matrix is symmetric)
                    data1 = df_filtered[df_filtered['model'] == model1].sort_values('fold')[metric]
                    data2 = df_filtered[df_filtered['model'] == model2].sort_values('fold')[metric]
                    
                    # Perform a paired t-test
                    if len(data1) == len(data2):  # Ensure the lengths match for paired t-test
                        t_stat, p_value = ttest_rel(data1, data2, alternative='less')
                        
                        # Populate the matrix
                        pval_matrix.loc[model1, model2] = p_value
                        pval_matrix.loc[model2, model1] = 1 - p_value

        print("P-Value Matrix:", input_type, metric)
        print(pval_matrix)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().